In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
shared_path = '/content/drive/Shareddrives/Baiying'
os.chdir(shared_path)
print("Working directory:", os.getcwd())

%cd Time-LLM

Mounted at /content/drive
Working directory: /content/drive/Shareddrives/Baiying
/content/drive/Shareddrives/Baiying/Time-LLM


In [ ]:
import pandas as pd
import numpy as np

result_df144_gpt2 = pd.read_csv('/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics/per_subject_horizons.csv')
# result_df96_llama = pd.read_csv('result_df2.csv')

In [ ]:
result_df144_gpt2.head()

,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl
0,2014,481,5.129536,7.188945,8.259652,12.070432,12.903422,18.112848,15.301770,20.258129
1,Subject 20,834,9.470576,13.389690,14.744808,20.265921,23.020927,31.054667,27.815025,36.858635
2,UoMGlucose2304,1470,12.239134,16.255600,19.452087,25.869877,31.263021,40.408318,38.680481,49.370609
3,P11,4203,9.780677,14.691290,15.596993,23.179062,24.254309,34.955303,29.583328,41.741734
4,P12,4218,8.000680,12.204821,12.492522,19.046652,19.162672,28.916431,23.617531,35.589825


# Task
Add a 'dataset_name' column to the `result_df144_gpt2` DataFrame by mapping each `subject_id` to its corresponding dataset subfolder, found within the '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset' directory (excluding the 'mixed' folder), and then display the head of the updated DataFrame.

## List Dataset Subfolders

### Subtask:
Recursively list all subdirectories within the path '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset', excluding the 'mixed' folder. These subdirectories represent the different datasets.


**Reasoning**:
First, I'll define the base directory for the datasets and then use `os.listdir` to get all entries. After that, I'll filter out files and the 'mixed' folder to get a list of dataset subfolders.



In [ ]:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset'

dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]

print("Dataset Subfolders:")
for folder in dataset_subfolders:
    print(folder)

Dataset Subfolders:
BIG_IDEA_LAB
ShanghaiT1DM
ShanghaiT2DM
UCHTT1DM
CGMacros
1_Hall2018
2_D1NAMO
3_Colas2019
14_HUPA-UCM
19_AZT1D
17_T1DM-UOM
18_Bris-T1D Open


**Reasoning**:
To map `subject_id`s to their respective dataset names, I will iterate through each `dataset_subfolder` identified previously. For each dataset, I will recursively search its directory structure for files or subfolders that exactly match any of the `subject_id`s from `result_df144_gpt2`, accounting for common variations like file extensions or spaces. This will build a mapping which will then be used to add the `dataset_name` column to the DataFrame, and finally display the head of the updated DataFrame.



In [ ]:
import os

dataset_files_map = {}

# Iterate through each dataset subfolder
for dataset_folder in dataset_subfolders:
    current_dataset_path = os.path.join(base_dataset_path, dataset_folder)
    files_in_dataset = []

    # Recursively walk through the current dataset's directory
    for root, _, files in os.walk(current_dataset_path):
        for file in files:
            # Normalize file name: convert to lowercase and remove extension
            file_name_without_extension = os.path.splitext(file)[0].lower()
            files_in_dataset.append(file_name_without_extension)

    # Store the normalized file names in the dictionary
    dataset_files_map[dataset_folder] = files_in_dataset

print("Dataset Files Map Generated. Displaying a sample (first few entries of each dataset):")
for dataset, files in dataset_files_map.items():
    print(f"  {dataset}: {files[:5]}... ({len(files)} files total)")

Dataset Files Map Generated. Displaying a sample (first few entries of each dataset):
  BIG_IDEA_LAB: ['dexcom_001', 'dexcom_002', 'dexcom_003', 'dexcom_004', 'dexcom_005']... (16 files total)
  ShanghaiT1DM: ['1001', '1002', '1004', '1003', '1005']... (12 files total)
  ShanghaiT2DM: ['2000', '2001', '2002', '2003', '2004']... (100 files total)
  UCHTT1DM: ['ht_01', 'ht_02', 'ht_03', 'ht_04', 'ht_05']... (20 files total)
  CGMacros: ['cgmacros-001', 'cgmacros-002', 'cgmacros-003', 'cgmacros-004', 'cgmacros-005']... (45 files total)
  1_Hall2018: ['2133-001', '2133-002', '2133-004', '2133-006', '2133-008']... (30 files total)
  2_D1NAMO: ['002', '001', '004', '003', '005']... (9 files total)
  3_Colas2019: ['case  1', 'case  10', 'case  100', 'case  101', 'case  102']... (208 files total)
  14_HUPA-UCM: ['hupa0010p', 'hupa0002p', 'hupa0004p', 'hupa0009p', 'hupa0007p']... (25 files total)
  19_AZT1D: ['subject 1', 'subject 10', 'subject 11', 'subject 12', 'subject 13']... (25 files tota

# Task
Get all file names from each dataset subfolder, excluding 'mixed', normalize them by converting to lowercase and removing extensions, and store them in a dictionary where keys are dataset names and values are lists of normalized file names.

In [ ]:
import pandas as pd

subject_to_dataset_map = {}
# Use a set for faster lookup of found subjects
found_subjects = set()

unique_subject_ids = result_df144_gpt2['subject_id'].astype(str).str.strip().unique()

# Create a dictionary of potential patterns for each subject_id to handle variations
potential_subject_patterns = {}
for sub_id in unique_subject_ids:
    s_id_str = str(sub_id).strip()
    patterns_base = [
        s_id_str,  # exact match
        s_id_str.replace(' ', '_'),  # spaces to underscores
        s_id_str.replace('-', '_'),  # hyphens to underscores (common in file names)
        s_id_str.replace('.', '')    # remove periods (e.g., for versions)
    ]
    # Add lowercase versions of all base patterns
    potential_subject_patterns[sub_id] = list(set(patterns_base + [p.lower() for p in patterns_base]))

print(f"Attempting to map {len(unique_subject_ids)} unique subject IDs to dataset subfolders...")

# Iterate through each dataset in the map
for dataset_name, normalized_files_list in dataset_files_map.items():
    # Convert the list to a set for faster lookup if it's large
    normalized_files_set = set(normalized_files_list)

    # Iterate through each unique subject ID
    for sub_id_original, patterns_to_check in potential_subject_patterns.items():
        if sub_id_original in found_subjects:  # Skip if already mapped
            continue

        # Check if any of the subject's patterns are in the current dataset's file list
        for pattern in patterns_to_check:
            if pattern in normalized_files_set:
                subject_to_dataset_map[sub_id_original] = dataset_name
                found_subjects.add(sub_id_original)
                break  # Found for this subject, move to the next subject

    # Optional: early exit if all subjects are found
    if len(found_subjects) == len(unique_subject_ids):
        print("All subjects mapped. Exiting dataset iteration early.")
        break

print(f"Mapped {len(found_subjects)} out of {len(unique_subject_ids)} subjects.")

# Add the 'dataset_name' column to result_df144_gpt2 using the created map
# Use .astype(str).str.strip() to ensure consistency with how unique_subject_ids were processed
result_df144_gpt2['dataset_name'] = result_df144_gpt2['subject_id'].astype(str).str.strip().map(subject_to_dataset_map)

# Report any unmapped subjects
unmapped_subjects = result_df144_gpt2[result_df144_gpt2['dataset_name'].isnull()]['subject_id'].unique()
if len(unmapped_subjects) > 0:
    print(f"Warning: {len(unmapped_subjects)} subjects could not be mapped to any dataset. Examples: {unmapped_subjects[:5]}...")
else:
    print("All subjects successfully mapped.")

# Display the head of the updated DataFrame
display(result_df144_gpt2.head())

Attempting to map 284 unique subject IDs to dataset subfolders...
All subjects mapped. Exiting dataset iteration early.
Mapped 284 out of 284 subjects.
All subjects successfully mapped.


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,2014,481,5.129536,7.188945,8.259652,12.070432,12.903422,18.112848,15.301770,20.258129,ShanghaiT2DM
1,Subject 20,834,9.470576,13.389690,14.744808,20.265921,23.020927,31.054667,27.815025,36.858635,19_AZT1D
2,UoMGlucose2304,1470,12.239134,16.255600,19.452087,25.869877,31.263021,40.408318,38.680481,49.370609,17_T1DM-UOM
3,P11,4203,9.780677,14.691290,15.596993,23.179062,24.254309,34.955303,29.583328,41.741734,18_Bris-T1D Open
4,P12,4218,8.000680,12.204821,12.492522,19.046652,19.162672,28.916431,23.617531,35.589825,18_Bris-T1D Open


In [ ]:
result_df144_gpt2

,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,2014,481,5.129536,7.188945,8.259652,12.070432,12.903422,18.112848,15.301770,20.258129,ShanghaiT2DM
1,Subject 20,834,9.470576,13.389690,14.744808,20.265921,23.020927,31.054667,27.815025,36.858635,19_AZT1D
2,UoMGlucose2304,1470,12.239134,16.255600,19.452087,25.869877,31.263021,40.408318,38.680481,49.370609,17_T1DM-UOM
3,P11,4203,9.780677,14.691290,15.596993,23.179062,24.254309,34.955303,29.583328,41.741734,18_Bris-T1D Open
4,P12,4218,8.000680,12.204821,12.492522,19.046652,19.162672,28.916431,23.617531,35.589825,18_Bris-T1D Open
...,...,...,...,...,...,...,...,...,...,...,...
279,2042,27,7.959016,9.793193,13.704017,16.983122,21.342697,25.680599,24.155449,28.351215,ShanghaiT2DM
280,2006,43,5.444734,8.092224,8.371539,12.390098,11.423732,17.716040,13.544275,20.655972,ShanghaiT2DM
281,002,17,30.617762,34.974064,49.328968,54.708488,78.126617,90.735092,100.733101,113.598846,2_D1NAMO
282,005,7,11.490624,13.914392,18.609936,21.449682,30.213556,34.412285,36.412483,40.794556,2_D1NAMO


In [ ]:
result_df144_gpt2 = result_df144_gpt2.sort_values(by=['dataset_name', 'subject_id']).reset_index(drop=True)
result_df144_gpt2

,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,HUPA0001P,220,14.865093,19.475784,23.378180,31.120819,36.869984,48.054474,46.971790,58.466087,14_HUPA-UCM
1,HUPA0002P,159,12.469090,17.317364,18.873333,27.778154,31.669846,44.647072,41.770973,57.184284,14_HUPA-UCM
2,HUPA0003P,198,11.000687,13.718702,18.510715,23.207954,34.059872,41.125095,45.424141,54.023922,14_HUPA-UCM
3,HUPA0004P,159,13.180820,17.968822,21.333946,28.802887,32.570953,45.265629,41.083828,58.437435,14_HUPA-UCM
4,HUPA0005P,204,7.848307,10.377975,12.579181,16.743513,19.623844,26.113077,24.567465,32.370895,14_HUPA-UCM
...,...,...,...,...,...,...,...,...,...,...,...
279,T1DM_04,66,12.843482,18.620768,20.942883,28.600819,33.775082,43.992805,44.049614,55.175304,UCHTT1DM
280,T1DM_05,56,9.504100,11.752862,13.531695,17.138186,23.042402,28.320255,31.158516,36.850883,UCHTT1DM
281,T1DM_06,41,15.774385,18.940752,24.770870,31.440174,45.890549,54.124840,55.048901,64.776093,UCHTT1DM
282,T1DM_07,31,8.362164,10.058887,11.847362,14.104342,18.065344,21.182949,24.653509,27.265999,UCHTT1DM


In [ ]:
summary_df = result_df144_gpt2.groupby('dataset_name').agg(
    avg_MAE_15min=('MAE_15min_mgdl', 'mean'),
    avg_RMSE_15min=('RMSE_15min_mgdl', 'mean'),
    avg_MAE_30min=('MAE_30min_mgdl', 'mean'),
    avg_RMSE_30min=('RMSE_30min_mgdl', 'mean'),
    avg_MAE_60min=('MAE_60min_mgdl', 'mean'),
    avg_RMSE_60min=('RMSE_60min_mgdl', 'mean'),
    avg_MAE_90min=('MAE_90min_mgdl', 'mean'),
    avg_RMSE_90min=('RMSE_90min_mgdl', 'mean')
).reset_index()

display(summary_df)

,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,14_HUPA-UCM,11.173271,14.863864,17.910070,23.987523,29.560181,38.638050,37.979523,48.498745
1,17_T1DM-UOM,12.136575,16.561397,19.039109,25.945939,28.946716,38.280264,34.805246,45.108901
2,18_Bris-T1D Open,9.504491,14.442215,14.927756,22.693003,23.199571,34.331758,28.265777,41.030116
3,19_AZT1D,11.342130,15.403662,17.263818,23.362078,26.280352,34.533597,31.554625,40.982625
4,2_D1NAMO,14.257180,18.166083,24.105858,29.117270,38.067363,45.051331,49.045645,56.543281
5,BIG_IDEA_LAB,7.754646,11.369164,11.246401,16.688330,14.327500,21.006025,15.680642,21.973470
6,CGMacros,8.630512,12.373543,12.407320,18.030894,15.773886,22.747449,18.150317,25.176284
7,ShanghaiT1DM,9.275657,11.757175,15.235501,19.342397,25.057965,31.803609,33.229817,41.634522
8,ShanghaiT2DM,6.943330,9.386716,11.157848,15.549766,17.700366,24.387912,22.279005,29.824129
9,UCHTT1DM,9.158476,13.054389,12.992642,17.934245,17.222755,22.521058,20.141194,25.189465


In [ ]:
import os

output_dir = '/content/drive/Shareddrives/Baiying/Time-LLM/results'

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save result_df144_gpt2
result_df144_gpt2_path = os.path.join(output_dir, 'result_df144_gpt2.csv')
result_df144_gpt2.to_csv(result_df144_gpt2_path, index=False)
print(f"result_df144_gpt2 saved to {result_df144_gpt2_path}")

# Save summary_df
summary_df_path = os.path.join(output_dir, 'summary_df_144_gpt2.csv')
summary_df.to_csv(summary_df_path, index=False)
print(f"summary_df saved to {summary_df_path}")

result_df144_gpt2 saved to /content/drive/Shareddrives/Baiying/Time-LLM/results/result_df144_gpt2.csv
summary_df saved to /content/drive/Shareddrives/Baiying/Time-LLM/results/summary_df_144_gpt2.csv


# Function

In [2]:
import os
import pandas as pd

def process_results_csv(csv_file_path, base_dataset_path, dataset_subfolders):
    """
    Processes a CSV file containing subject-level results, maps subject IDs to datasets,
    and generates a summary table of averaged MAE and RMSE per dataset.

    Args:
        csv_file_path (str): The path to the input CSV file.
        base_dataset_path (str): The base directory containing dataset subfolders.
        dataset_subfolders (list): A list of dataset subfolder names to consider for mapping.

    Returns:
        tuple: A tuple containing two pandas DataFrames:
               - result_df_processed: The original DataFrame with an added 'dataset_name' column, sorted.
               - summary_df: A summary DataFrame with averaged MAE and RMSE per dataset.
    """
    print(f"\nProcessing file: {csv_file_path}")

    # 1. Load the CSV file
    result_df = pd.read_csv(csv_file_path)

    # 2. Create dataset_files_map for this run
    dataset_files_map = {}
    for dataset_folder in dataset_subfolders:
        current_dataset_path = os.path.join(base_dataset_path, dataset_folder)
        files_in_dataset = []
        if os.path.isdir(current_dataset_path): # Ensure it's a directory
            for root, _, files in os.walk(current_dataset_path):
                for file in files:
                    file_name_without_extension = os.path.splitext(file)[0].lower()
                    files_in_dataset.append(file_name_without_extension)
        dataset_files_map[dataset_folder] = files_in_dataset

    # 3. Map Subject ID to Dataset Name
    subject_to_dataset_map = {}
    found_subjects = set()
    unique_subject_ids = result_df['subject_id'].astype(str).str.strip().unique()

    potential_subject_patterns = {}
    for sub_id in unique_subject_ids:
        s_id_str = str(sub_id).strip()
        patterns_base = [
            s_id_str,
            s_id_str.replace(' ', '_'),
            s_id_str.replace('-', '_'),
            s_id_str.replace('.', '')
        ]
        potential_subject_patterns[sub_id] = list(set(patterns_base + [p.lower() for p in patterns_base]))

    print(f"Attempting to map {len(unique_subject_ids)} unique subject IDs...")
    for dataset_name, normalized_files_list in dataset_files_map.items():
        normalized_files_set = set(normalized_files_list)
        for sub_id_original, patterns_to_check in potential_subject_patterns.items():
            if sub_id_original in found_subjects:
                continue
            for pattern in patterns_to_check:
                if pattern in normalized_files_set:
                    subject_to_dataset_map[sub_id_original] = dataset_name
                    found_subjects.add(sub_id_original)
                    break
        if len(found_subjects) == len(unique_subject_ids):
            print("All subjects mapped. Exiting dataset iteration early.")
            break

    result_df['dataset_name'] = result_df['subject_id'].astype(str).str.strip().map(subject_to_dataset_map)

    unmapped_subjects = result_df[result_df['dataset_name'].isnull()]['subject_id'].unique()
    if len(unmapped_subjects) > 0:
        print(f"Warning: {len(unmapped_subjects)} subjects could not be mapped. Examples: {unmapped_subjects[:5]}...")
    else:
        print("All subjects successfully mapped.")

    # Sort the dataframe
    result_df_processed = result_df.sort_values(by=['dataset_name', 'subject_id']).reset_index(drop=True)

    # 4. Calculate summary_df
    summary_df = result_df_processed.groupby('dataset_name').agg(
        avg_MAE_15min=('MAE_15min_mgdl', 'mean'),
        avg_RMSE_15min=('RMSE_15min_mgdl', 'mean'),
        avg_MAE_30min=('MAE_30min_mgdl', 'mean'),
        avg_RMSE_30min=('RMSE_30min_mgdl', 'mean'),
        avg_MAE_60min=('MAE_60min_mgdl', 'mean'),
        avg_RMSE_60min=('RMSE_60min_mgdl', 'mean'),
        avg_MAE_90min=('MAE_90min_mgdl', 'mean'),
        avg_RMSE_90min=('RMSE_90min_mgdl', 'mean')
    ).reset_index()

    return result_df_processed, summary_df

In [ ]:
# Example usage of the function with the original file:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset'
main_csv_path = '/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_few_shot_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_fewshot/test_metrics/per_subject_horizons.csv'
dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]
# Ensure base_dataset_path and dataset_subfolders are defined from previous steps
# (If not, you'd need to define them here, e.g., base_dataset_path = '...', dataset_subfolders = [...] )

processed_df_gpt2, summary_table_gpt2 = process_results_csv(main_csv_path, base_dataset_path, dataset_subfolders)

print("\nProcessed DataFrame (first 5 rows):")
display(processed_df_gpt2.head())

print("\nSummary Table:")
display(summary_table_gpt2)


Processing file: /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_few_shot_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_fewshot/test_metrics/per_subject_horizons.csv
Attempting to map 336 unique subject IDs...
All subjects mapped. Exiting dataset iteration early.
All subjects successfully mapped.

Processed DataFrame (first 5 rows):


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,HUPA0001P,659,20.902388,27.992907,30.146786,40.016785,43.124229,55.302517,50.433075,63.604355,14_HUPA-UCM
1,HUPA0002P,476,17.383646,25.169159,24.473719,36.426922,34.819679,49.408646,42.988972,59.065742,14_HUPA-UCM
2,HUPA0003P,593,16.687838,20.609901,24.351725,30.510250,39.375683,47.791653,47.413193,56.634678,14_HUPA-UCM
3,HUPA0004P,476,17.340153,24.635181,25.264431,35.872036,37.266975,53.917324,43.676815,62.915188,14_HUPA-UCM
4,HUPA0005P,611,10.268481,13.950452,14.394892,19.634359,21.412111,28.014868,25.915258,33.692997,14_HUPA-UCM



Summary Table:


,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,14_HUPA-UCM,15.256217,20.270571,21.928737,29.252372,33.240085,43.096236,39.298372,50.264906
1,17_T1DM-UOM,15.981895,21.555446,22.384459,30.126987,31.019613,40.902235,35.634412,46.313357
2,18_Bris-T1D Open,12.625507,19.065130,17.854514,26.853240,25.104398,37.042134,28.974770,42.204532
3,19_AZT1D,14.741371,19.673258,20.478281,27.230195,28.311906,37.114305,32.351763,42.157120
4,1_Hall2018,8.621664,12.051299,11.375576,15.660669,13.714583,18.688347,15.376602,20.523315
5,2_D1NAMO,19.007164,23.401601,28.200213,34.416271,43.942848,51.058644,51.499028,59.070120
6,BIG_IDEA_LAB,9.389630,13.698909,12.201960,17.709424,14.388986,20.980560,15.742704,22.145237
7,CGMacros,10.372312,14.898379,13.510708,19.373596,16.153333,23.325856,18.354538,25.528556
8,ShanghaiT1DM,12.692430,16.274407,18.329560,23.602441,28.273073,35.898871,34.602830,43.453941
9,ShanghaiT2DM,9.523897,13.129098,13.536441,18.711161,19.319951,26.640060,23.047523,30.954761


In [ ]:
import os

# Extract the directory of the main CSV file
output_metrics_dir = os.path.dirname(main_csv_path)

# Create the directory if it doesn't exist
os.makedirs(output_metrics_dir, exist_ok=True)

# Save processed_df_gpt2
processed_df_gpt2_path = os.path.join(output_metrics_dir, 'processed_per_subject_horizons_few_shot.csv')
processed_df_gpt2.to_csv(processed_df_gpt2_path, index=False)
print(f"Processed DataFrame saved to {processed_df_gpt2_path}")

# Save summary_table_gpt2
summary_table_gpt2_path = os.path.join(output_metrics_dir, 'summary_per_dataset_few_shot.csv')
summary_table_gpt2.to_csv(summary_table_gpt2_path, index=False)
print(f"Summary table saved to {summary_table_gpt2_path}")

Processed DataFrame saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_few_shot_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_fewshot/test_metrics/processed_per_subject_horizons_few_shot.csv
Summary table saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_few_shot_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_fewshot/test_metrics/summary_per_dataset_few_shot.csv


In [ ]:
# Example usage of the function with the original file:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset'
main_csv_path = '/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_llama_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_LLaMA_0-GlucoseTrain_LLaMA_7to1/test_metrics/per_subject_horizons.csv'
dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]
# Ensure base_dataset_path and dataset_subfolders are defined from previous steps
# (If not, you'd need to define them here, e.g., base_dataset_path = '...', dataset_subfolders = [...] )

processed_df_llama, summary_table_llama = process_results_csv(main_csv_path, base_dataset_path, dataset_subfolders)

print("\nProcessed DataFrame (first 5 rows):")
display(processed_df_llama.head())

print("\nSummary Table:")
display(summary_table_llama)

# Extract the directory of the main CSV file
output_metrics_dir = os.path.dirname(main_csv_path)

# Create the directory if it doesn't exist
os.makedirs(output_metrics_dir, exist_ok=True)

# Save processed_df_gpt2
processed_df_llama_path = os.path.join(output_metrics_dir, 'processed_per_subject_horizons.csv')
processed_df_llama.to_csv(processed_df_llama_path, index=False)
print(f"Processed DataFrame saved to {processed_df_llama_path}")

# Save summary_table_gpt2
summary_table_llama_path = os.path.join(output_metrics_dir, 'summary_per_dataset.csv')
summary_table_llama.to_csv(summary_table_llama_path, index=False)
print(f"Summary table saved to {summary_table_llama_path}")


Processing file: /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_llama_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_LLaMA_0-GlucoseTrain_LLaMA_7to1/test_metrics/per_subject_horizons.csv
Attempting to map 284 unique subject IDs...
All subjects mapped. Exiting dataset iteration early.
All subjects successfully mapped.

Processed DataFrame (first 5 rows):


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,HUPA0001P,220,16.627977,22.315451,24.519548,33.497257,37.995411,49.687469,47.589840,60.572521,14_HUPA-UCM
1,HUPA0002P,159,12.791978,18.748068,18.809738,28.206730,30.158186,44.613728,39.573154,56.097626,14_HUPA-UCM
2,HUPA0003P,198,12.608635,16.056078,19.655920,25.237402,34.192249,42.120777,45.407604,54.489445,14_HUPA-UCM
3,HUPA0004P,159,13.824431,21.100088,20.303123,30.732492,34.020233,49.338169,43.003399,63.071762,14_HUPA-UCM
4,HUPA0005P,204,9.053688,11.580091,13.097636,17.187433,19.589930,26.132938,24.929956,32.454365,14_HUPA-UCM



Summary Table:


,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,14_HUPA-UCM,12.283298,16.788394,18.192060,24.950259,30.033694,39.766332,38.158285,49.321876
1,17_T1DM-UOM,13.428438,18.409987,19.644779,26.995394,29.351626,39.066394,35.048093,45.837396
2,18_Bris-T1D Open,10.450984,16.118345,15.315875,23.604935,23.478581,35.104569,28.412057,41.724699
3,19_AZT1D,12.538316,16.991336,18.042013,24.437937,26.778468,35.392052,31.944195,41.767862
4,2_D1NAMO,17.047620,20.879870,25.079034,31.024368,39.920333,46.789992,49.888982,57.509186
5,BIG_IDEA_LAB,8.216728,12.176639,11.374502,17.121581,14.436197,21.131006,15.411425,21.969541
6,CGMacros,9.388100,13.539540,12.620158,18.488338,15.945412,23.035029,18.092193,25.454798
7,ShanghaiT1DM,10.190849,13.286791,15.237271,19.903730,25.723722,33.006701,33.485592,42.510141
8,ShanghaiT2DM,7.706219,10.623383,11.426483,16.068303,17.904676,24.991857,22.423329,30.396728
9,UCHTT1DM,9.617243,13.701583,13.099205,18.311501,17.211437,22.621100,20.194973,25.344888


Processed DataFrame saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_llama_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_LLaMA_0-GlucoseTrain_LLaMA_7to1/test_metrics/processed_per_subject_horizons.csv
Summary table saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_llama_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_LLaMA_0-GlucoseTrain_LLaMA_7to1/test_metrics/summary_per_dataset.csv


In [ ]:
# Example usage of the function with the original file:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset'
main_csv_path = '/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_shorter_TimeLLM_Glucose_ftS_sl48_ll24_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_shorter/test_metrics/per_subject_horizons.csv'
dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]
# Ensure base_dataset_path and dataset_subfolders are defined from previous steps
# (If not, you'd need to define them here, e.g., base_dataset_path = '...', dataset_subfolders = [...] )

processed_df_llama, summary_table_llama = process_results_csv(main_csv_path, base_dataset_path, dataset_subfolders)

print("\nProcessed DataFrame (first 5 rows):")
display(processed_df_llama.head())

print("\nSummary Table:")
display(summary_table_llama)

# Extract the directory of the main CSV file
output_metrics_dir = os.path.dirname(main_csv_path)

# Create the directory if it doesn't exist
os.makedirs(output_metrics_dir, exist_ok=True)

# Save processed_df_gpt2
processed_df_llama_path = os.path.join(output_metrics_dir, 'processed_per_subject_horizons.csv')
processed_df_llama.to_csv(processed_df_llama_path, index=False)
print(f"Processed DataFrame saved to {processed_df_llama_path}")

# Save summary_table_gpt2
summary_table_llama_path = os.path.join(output_metrics_dir, 'summary_per_dataset.csv')
summary_table_llama.to_csv(summary_table_llama_path, index=False)
print(f"Summary table saved to {summary_table_llama_path}")


Processing file: /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_shorter_TimeLLM_Glucose_ftS_sl48_ll24_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_shorter/test_metrics/per_subject_horizons.csv
Attempting to map 287 unique subject IDs...
All subjects mapped. Exiting dataset iteration early.
All subjects successfully mapped.

Processed DataFrame (first 5 rows):


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,HUPA0001P,252,17.269947,22.050060,25.980906,33.916466,38.911129,51.382507,50.015057,64.095413,14_HUPA-UCM
1,HUPA0002P,191,8.627788,12.522798,15.345289,22.216366,27.253241,40.101536,36.034561,52.480515,14_HUPA-UCM
2,HUPA0003P,230,10.837124,13.593976,19.245958,24.123615,34.509487,43.633228,46.059669,57.627010,14_HUPA-UCM
3,HUPA0004P,191,9.900386,14.216654,17.465302,25.155056,28.650284,43.816559,37.404312,57.960880,14_HUPA-UCM
4,HUPA0005P,236,7.599450,9.805928,12.387903,16.097828,20.169895,25.956196,26.112715,32.468681,14_HUPA-UCM



Summary Table:


,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,14_HUPA-UCM,9.826273,13.479242,17.025045,23.031794,28.438977,38.435827,37.367172,49.355218
1,17_T1DM-UOM,11.347813,15.875499,18.523201,25.445084,29.069245,39.187238,35.474394,47.049705
2,18_Bris-T1D Open,8.712457,13.686022,14.339388,22.076546,22.955152,34.887208,28.358838,42.410875
3,19_AZT1D,10.902754,15.153108,17.163334,23.250677,26.869460,35.606278,32.815060,42.986443
4,2_D1NAMO,13.465986,17.848810,22.989810,30.011192,39.274624,49.879210,49.956298,62.936628
5,BIG_IDEA_LAB,7.613293,11.518452,11.139576,17.003011,15.022735,22.294793,16.629838,23.905884
6,CGMacros,8.826354,12.713916,12.765554,18.543182,17.312551,24.754956,19.996809,27.903088
7,ShanghaiT1DM,7.223711,9.453605,13.280927,17.272061,22.927679,29.823038,30.626956,39.453512
8,ShanghaiT2DM,6.012078,8.345666,10.437696,14.535677,17.111001,23.916407,21.877227,29.819972
9,UCHTT1DM,9.657720,13.926677,13.956511,19.783218,19.586281,26.592954,23.648665,30.875781


Processed DataFrame saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_shorter_TimeLLM_Glucose_ftS_sl48_ll24_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_shorter/test_metrics/processed_per_subject_horizons.csv
Summary table saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_shorter_TimeLLM_Glucose_ftS_sl48_ll24_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_shorter/test_metrics/summary_per_dataset.csv


In [ ]:
# Example usage of the function with the original file:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset'
main_csv_path = '/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/per_subject_horizons.csv'
dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]
# Ensure base_dataset_path and dataset_subfolders are defined from previous steps
# (If not, you'd need to define them here, e.g., base_dataset_path = '...', dataset_subfolders = [...] )

processed_df_llama, summary_table_llama = process_results_csv(main_csv_path, base_dataset_path, dataset_subfolders)

print("\nProcessed DataFrame (first 5 rows):")
display(processed_df_llama.head())

print("\nSummary Table:")
display(summary_table_llama)

# Extract the directory of the main CSV file
output_metrics_dir = os.path.dirname(main_csv_path)

# Create the directory if it doesn't exist
os.makedirs(output_metrics_dir, exist_ok=True)

# Save processed_df_gpt2
processed_df_llama_path = os.path.join(output_metrics_dir, 'processed_per_subject_horizons.csv')
processed_df_llama.to_csv(processed_df_llama_path, index=False)
print(f"Processed DataFrame saved to {processed_df_llama_path}")

# Save summary_table_gpt2
summary_table_llama_path = os.path.join(output_metrics_dir, 'summary_per_dataset.csv')
summary_table_llama.to_csv(summary_table_llama_path, index=False)
print(f"Summary table saved to {summary_table_llama_path}")


Processing file: /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/per_subject_horizons.csv
Attempting to map 525 unique subject IDs...
All subjects mapped. Exiting dataset iteration early.
All subjects successfully mapped.

Processed DataFrame (first 5 rows):


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,HUPA0001P,264,10.639984,15.372398,20.974298,29.524532,37.108948,50.658981,48.767883,63.542236,14_HUPA-UCM
1,HUPA0002P,203,5.500349,8.072974,11.803192,17.660952,23.391726,34.906620,32.722343,48.210220,14_HUPA-UCM
2,HUPA0003P,242,6.908622,8.843836,14.785915,18.728430,29.999441,37.813709,42.340141,52.718918,14_HUPA-UCM
3,HUPA0004P,203,4.939046,8.218683,11.585119,18.689625,23.957607,37.812355,32.877438,52.333080,14_HUPA-UCM
4,HUPA0005P,248,5.714240,7.402208,11.371640,14.645122,19.841932,25.464037,26.206526,32.937996,14_HUPA-UCM



Summary Table:


,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,14_HUPA-UCM,6.308367,9.322825,13.255808,18.835644,25.567223,35.060241,35.011152,47.066550
1,17_T1DM-UOM,8.564149,12.685231,16.242007,23.183064,28.255654,38.778567,35.949035,48.278641
2,18_Bris-T1D Open,6.582223,10.811802,12.394220,19.907096,22.074938,34.268062,28.532228,43.510835
3,19_AZT1D,9.039741,13.091901,15.826403,22.081152,26.960878,36.131193,34.363415,45.228511
4,1_Hall2018,10.753185,12.925789,19.732250,23.018451,27.842928,31.935298,28.035975,32.167250
5,2_D1NAMO,7.513975,11.192362,15.707702,22.593002,31.091601,42.105175,41.398036,54.763733
6,3_Colas2019,9.761891,12.481105,14.806137,17.807316,17.266816,21.327559,18.846543,22.724749
7,BIG_IDEA_LAB,5.801460,8.968835,10.559759,16.275355,15.731764,23.651833,17.391263,25.518468
8,CGMacros,7.307925,10.725720,12.570776,18.640480,19.036532,27.351404,22.259296,31.124108
9,ShanghaiT1DM,4.434843,6.047526,10.096702,13.757608,20.488864,27.464155,28.954006,38.314575


Processed DataFrame saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/processed_per_subject_horizons.csv
Summary table saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/summary_per_dataset.csv


In [ ]:
# Example usage of the function with the original file:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset'
main_csv_path = '/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/short_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/per_subject_horizons.csv'
dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]
# Ensure base_dataset_path and dataset_subfolders are defined from previous steps
# (If not, you'd need to define them here, e.g., base_dataset_path = '...', dataset_subfolders = [...] )

processed_df_llama, summary_table_llama = process_results_csv(main_csv_path, base_dataset_path, dataset_subfolders)

print("\nProcessed DataFrame (first 5 rows):")
display(processed_df_llama.head())

print("\nSummary Table:")
display(summary_table_llama)

# Extract the directory of the main CSV file
output_metrics_dir = os.path.dirname(main_csv_path)

# Create the directory if it doesn't exist
os.makedirs(output_metrics_dir, exist_ok=True)

# Save processed_df_gpt2
processed_df_llama_path = os.path.join(output_metrics_dir, 'processed_per_subject_horizons.csv')
processed_df_llama.to_csv(processed_df_llama_path, index=False)
print(f"Processed DataFrame saved to {processed_df_llama_path}")

# Save summary_table_gpt2
summary_table_llama_path = os.path.join(output_metrics_dir, 'summary_per_dataset.csv')
summary_table_llama.to_csv(summary_table_llama_path, index=False)
print(f"Summary table saved to {summary_table_llama_path}")


Processing file: /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/short_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/per_subject_horizons.csv
Attempting to map 525 unique subject IDs...
All subjects mapped. Exiting dataset iteration early.
All subjects successfully mapped.

Processed DataFrame (first 5 rows):


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,HUPA0001P,264,10.638708,15.371169,20.971634,29.522293,37.107281,50.658081,48.767853,63.542320,14_HUPA-UCM
1,HUPA0002P,203,5.498854,8.070874,11.801228,17.658447,23.389528,34.904076,32.720734,48.208748,14_HUPA-UCM
2,HUPA0003P,242,6.905819,8.841243,14.782942,18.725487,29.996857,37.811211,42.339218,52.717888,14_HUPA-UCM
3,HUPA0004P,203,4.936920,8.216132,11.582747,18.687101,23.955961,37.810154,32.876369,52.331631,14_HUPA-UCM
4,HUPA0005P,248,5.713704,7.401483,11.371176,14.644662,19.841345,25.463802,26.205980,32.937874,14_HUPA-UCM



Summary Table:


,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,14_HUPA-UCM,6.306665,9.321198,13.254092,18.833966,25.565766,35.058810,35.010431,47.065917
1,17_T1DM-UOM,8.563475,12.684731,16.241540,23.182865,28.255415,38.778759,35.948955,48.278756
2,18_Bris-T1D Open,6.581501,10.811143,12.393536,19.906558,22.074491,34.267854,28.532087,43.510824
3,19_AZT1D,9.039290,13.091668,15.825881,22.081038,26.960586,36.131384,34.363347,45.228686
4,1_Hall2018,10.754457,12.926281,19.733792,23.019694,27.844912,31.938070,28.037876,32.169573
5,2_D1NAMO,7.511330,11.189857,15.705100,22.590696,31.089881,42.103729,41.397677,54.763574
6,3_Colas2019,9.762845,12.482194,14.808008,17.809396,17.269119,21.330463,18.847643,22.726111
7,BIG_IDEA_LAB,5.801674,8.969061,10.560562,16.276432,15.732997,23.653492,17.391905,25.519342
8,CGMacros,7.308077,10.725936,12.571288,18.641308,19.037264,27.352527,22.259655,31.124648
9,ShanghaiT1DM,4.433177,6.045575,10.095001,13.755721,20.487134,27.462270,28.953085,38.313567


Processed DataFrame saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/short_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/processed_per_subject_horizons.csv
Summary table saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/short_term_forecast_Glucose_train_1h_TimeLLM_Glucose_ftS_sl12_ll6_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_1h/test_metrics/summary_per_dataset.csv


In [ ]:
"abba"[1:-1]

'bb'

In [ ]:
# Example usage of the function with the original file:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset'
main_csv_path = '/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl6_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics/per_subject_horizons.csv'
dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]
# Ensure base_dataset_path and dataset_subfolders are defined from previous steps
# (If not, you'd need to define them here, e.g., base_dataset_path = '...', dataset_subfolders = [...] )

processed_df_llama, summary_table_llama = process_results_csv(main_csv_path, base_dataset_path, dataset_subfolders)

print("\nProcessed DataFrame (first 5 rows):")
display(processed_df_llama.head())

print("\nSummary Table:")
display(summary_table_llama)

# Extract the directory of the main CSV file
output_metrics_dir = os.path.dirname(main_csv_path)

# Create the directory if it doesn't exist
os.makedirs(output_metrics_dir, exist_ok=True)

# Save processed_df_gpt2
processed_df_llama_path = os.path.join(output_metrics_dir, 'processed_per_subject_horizons.csv')
processed_df_llama.to_csv(processed_df_llama_path, index=False)
print(f"Processed DataFrame saved to {processed_df_llama_path}")

# Save summary_table_gpt2
summary_table_llama_path = os.path.join(output_metrics_dir, 'summary_per_dataset.csv')
summary_table_llama.to_csv(summary_table_llama_path, index=False)
print(f"Summary table saved to {summary_table_llama_path}")


Processing file: /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl6_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics/per_subject_horizons.csv
Attempting to map 284 unique subject IDs...
All subjects mapped. Exiting dataset iteration early.
All subjects successfully mapped.

Processed DataFrame (first 5 rows):


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,HUPA0001P,68,15.120321,19.850939,23.138386,31.126598,NaN,NaN,NaN,NaN,14_HUPA-UCM
1,HUPA0002P,49,10.274449,13.884756,14.820626,22.711771,NaN,NaN,NaN,NaN,14_HUPA-UCM
2,HUPA0003P,61,10.496746,13.233643,18.697708,23.542353,NaN,NaN,NaN,NaN,14_HUPA-UCM
3,HUPA0004P,49,11.025877,15.400781,20.250650,26.901165,NaN,NaN,NaN,NaN,14_HUPA-UCM
4,HUPA0005P,63,8.153665,10.624302,14.636031,19.377403,NaN,NaN,NaN,NaN,14_HUPA-UCM



Summary Table:


,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,14_HUPA-UCM,10.117306,13.444838,16.887383,22.353984,NaN,NaN,NaN,NaN
1,17_T1DM-UOM,11.119575,15.062687,18.311067,25.124865,NaN,NaN,NaN,NaN
2,18_Bris-T1D Open,8.730878,13.068935,14.187752,21.561882,NaN,NaN,NaN,NaN
3,19_AZT1D,10.591001,14.075287,16.507055,22.084673,NaN,NaN,NaN,NaN
4,2_D1NAMO,10.197991,12.396733,19.833814,24.070885,NaN,NaN,NaN,NaN
5,BIG_IDEA_LAB,7.339478,10.537621,11.637649,16.979060,NaN,NaN,NaN,NaN
6,CGMacros,8.246938,11.437215,12.200718,17.747785,NaN,NaN,NaN,NaN
7,ShanghaiT1DM,8.017038,9.972034,14.013485,17.683373,NaN,NaN,NaN,NaN
8,ShanghaiT2DM,6.343999,8.312216,10.602626,14.666408,NaN,NaN,NaN,NaN
9,UCHTT1DM,9.579144,12.687262,14.210623,18.601865,NaN,NaN,NaN,NaN


Processed DataFrame saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl6_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics/processed_per_subject_horizons.csv
Summary table saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl6_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics/summary_per_dataset.csv


In [5]:
# Example usage of the function with the original file:
base_dataset_path = '/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset/controlled_datasets'
main_csv_path = '/content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics-controlled/per_subject_horizons.csv'
dataset_subfolders = [
    f for f in os.listdir(base_dataset_path)
    if os.path.isdir(os.path.join(base_dataset_path, f)) and f != 'mixed'
]
# Ensure base_dataset_path and dataset_subfolders are defined from previous steps
# (If not, you'd need to define them here, e.g., base_dataset_path = '...', dataset_subfolders = [...] )

processed_df_llama, summary_table_llama = process_results_csv(main_csv_path, base_dataset_path, dataset_subfolders)

print("\nProcessed DataFrame (first 5 rows):")
display(processed_df_llama.head())

print("\nSummary Table:")
display(summary_table_llama)

# Extract the directory of the main CSV file
output_metrics_dir = os.path.dirname(main_csv_path)

# Create the directory if it doesn't exist
os.makedirs(output_metrics_dir, exist_ok=True)

# Save processed_df_gpt2
processed_df_llama_path = os.path.join(output_metrics_dir, 'processed_per_subject_horizons_controlled.csv')
processed_df_llama.to_csv(processed_df_llama_path, index=False)
print(f"Processed DataFrame saved to {processed_df_llama_path}")

# Save summary_table_gpt2
summary_table_llama_path = os.path.join(output_metrics_dir, 'summary_per_dataset_controlled.csv')
summary_table_llama.to_csv(summary_table_llama_path, index=False)
print(f"Summary table saved to {summary_table_llama_path}")


Processing file: /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics-controlled/per_subject_horizons.csv
Attempting to map 2101 unique subject IDs...

Processed DataFrame (first 5 rows):


,subject_id,num_windows,MAE_15min_mgdl,RMSE_15min_mgdl,MAE_30min_mgdl,RMSE_30min_mgdl,MAE_60min_mgdl,RMSE_60min_mgdl,MAE_90min_mgdl,RMSE_90min_mgdl,dataset_name
0,LIB193263,12004,10.611241,14.180233,17.354193,23.119539,28.252554,36.975311,36.295940,46.944622,11_T1DiabetesGranada
1,LIB193264,5307,10.834572,15.212785,17.738129,24.272333,29.051489,38.080898,37.142754,47.900158,11_T1DiabetesGranada
2,LIB193265,9257,9.622683,13.652442,15.318868,21.817547,24.680336,34.221447,31.459354,42.687283,11_T1DiabetesGranada
3,LIB193266,8856,12.926258,18.105806,20.510605,28.962751,30.621746,42.430336,35.920605,48.493187,11_T1DiabetesGranada
4,LIB193267,10877,13.865680,19.453739,22.468382,30.997313,36.102165,48.226116,45.504604,59.447765,11_T1DiabetesGranada



Summary Table:


,dataset_name,avg_MAE_15min,avg_RMSE_15min,avg_MAE_30min,avg_RMSE_30min,avg_MAE_60min,avg_RMSE_60min,avg_MAE_90min,avg_RMSE_90min
0,11_T1DiabetesGranada,12.221843,17.016064,19.847299,27.155436,31.863139,42.089224,40.044784,51.757296
1,12_AI-READI,8.487715,11.973241,12.075519,17.190061,15.645794,21.868453,17.786521,23.998374
2,5_T1DEXI,11.815449,16.077102,18.219733,24.750816,27.664321,36.563924,33.407616,43.355315
3,6_T1DEXIP,14.320368,18.461548,23.170540,30.050014,35.821018,46.369682,43.276360,56.395561
4,8_DiaTrend,14.551118,19.885455,22.486724,30.466653,34.840614,45.687950,42.959579,55.260461
5,OhioT1DM,11.221931,15.500027,17.544958,23.897940,27.812290,36.755649,35.146110,45.484141


Processed DataFrame saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics-controlled/processed_per_subject_horizons_controlled.csv
Summary table saved to /content/drive/Shareddrives/Baiying/Time-LLM/checkpoints/long_term_forecast_Glucose_train_main_TimeLLM_Glucose_ftS_sl144_ll72_pl18_dm16_nh8_el2_dl1_df32_fc1_ebtimeF_GlucoseTrain_0-GlucoseTrain_7to1/test_metrics-controlled/summary_per_dataset_controlled.csv
